In [1]:
# Importar librerías necesarias
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os
import time

# Configurar Kaggle API y descargar datos
print("Por favor, sube el archivo kaggle.json")
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configurado correctamente")
else:
    raise FileNotFoundError("No se subió kaggle.json. Por favor, sube el archivo y vuelve a intentarlo.")

# Instalar Kaggle CLI
!pip install kaggle --quiet

# Descargar y extraer datos
start_time = time.time()
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
if os.path.exists("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"):
    with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
        zip_ref.extractall("datos")
    print("Datos extraídos en datos/")
else:
    raise FileNotFoundError("Fallo al descargar el ZIP. Verifica tus credenciales de Kaggle o tu conexión a internet.")
print(f"Tiempo de descarga y extracción: {(time.time() - start_time) / 60:.2f} minutos")

# Verificar archivos en datos/
!ls -lh datos/

# Cargar datos
try:
    train = pd.read_csv('datos/train.csv', engine='python', encoding='utf-8')
    print("train.csv cargado correctamente")
except Exception as e:
    print(f"Error al cargar train.csv: {e}")
    raise

# Verificar columnas iniciales
print('Columnas iniciales:', train.columns.tolist())
print('Tipos de datos iniciales:\n', train.dtypes)

# Eliminar columna duplicada si existe
if 'FAMI_TIENEINTERNET.1' in train.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1")
    train = train.drop('FAMI_TIENEINTERNET.1', axis=1)

# Estandarizar valores para evitar inconsistencias
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')

# Manejar valores faltantes
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no pagó matrícula')
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna('0')
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna('No sabe')
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna('No sabe')
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].fillna('No')
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].fillna('No')
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].fillna('No')
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].fillna('No')
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna('No')
train['ESTU_PRIVADO_LIBERTAD'] = train['ESTU_PRIVADO_LIBERTAD'].fillna('N')

# Definir mapeo para codificación ordinal
valormatricula_map = {
    'no pagó matrícula': 0,
    'menos de 500 mil': 1,
    'entre 500 mil y menos de 1 millón': 2,
    'entre 1 millón y menos de 2.5 millones': 3,
    'entre 2.5 millones y menos de 4 millones': 4,
    'entre 4 millones y menos de 5.5 millones': 5,
    'entre 5.5 millones y menos de 7 millones': 6,
    'más de 7 millones': 7
}
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)

horastrabajo_map = {
    '0': 0,
    'menos de 10 horas': 1,
    'entre 11 y 20 horas': 2,
    'entre 21 y 30 horas': 3,
    'más de 30 horas': 4
}
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)

# Aplicar Label Encoding a categóricas
label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE']
le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    le_dict[col] = le

# Aplicar One-Hot Encoding a todas las columnas categóricas/binarias
one_hot_cols = ['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_TIENEINTERNET',
                'FAMI_TIENECOMPUTADOR', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRIVADO_LIBERTAD']
train = pd.get_dummies(train, columns=one_hot_cols, drop_first=True)

# Normalizar columnas numéricas
num_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
scaler = MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])

# Eliminar columnas duplicadas
train = train.loc[:, ~train.columns.duplicated()]

# Verificar datos después de preprocesar
print('Columnas después de preprocesar:', train.columns.tolist())
print('Tipos de datos después de preprocesar:\n', train.dtypes)
for col in train.columns:
    if train[col].dtype == 'object' and col not in ['ID', 'RENDIMIENTO_GLOBAL']:
        print(f"Columna con strings: {col}, valores: {train[col].unique()}")

# Guardar el archivo preprocesado
train.to_csv('datos/train_preprocessed.csv', index=False)
print('train_preprocessed.csv guardado')

# Verificar que el archivo se guardó
if os.path.exists('datos/train_preprocessed.csv'):
    print(f"train_preprocessed.csv existe, tamaño: {os.path.getsize('datos/train_preprocessed.csv')} bytes")
else:
    raise FileNotFoundError("Error: train_preprocessed.csv no se guardó correctamente")

Por favor, sube el archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.07GB/s]
Datos extraídos en datos/
Tiempo de descarga y extracción: 0.05 minutos
total 199M
-rw-r--r-- 1 root root 4.5M Jul  3 03:06 submission_example.csv
-rw-r--r-- 1 root root  57M Jul  3 03:06 test.csv
-rw-r--r-- 1 root root 138M Jul  3 03:06 train.csv
train.csv cargado correctamente
Columnas iniciales: ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4']
Tipos de datos iniciales:
 ID                                  int64
PERIODO                             int64
ESTU_PRGM_ACADEM

In [2]:
# Importar librerías necesarias
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import os
import time

# Configurar Kaggle API y descargar datos
print("Por favor, sube el archivo kaggle.json")
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configurado correctamente")
else:
    raise FileNotFoundError("No se subió kaggle.json")

!pip install kaggle --quiet
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
if os.path.exists("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"):
    with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
        zip_ref.extractall("datos")
    print("Datos extraídos en datos/")
    if os.path.exists('datos/test.csv'):
        print(f"test.csv encontrado, tamaño: {os.path.getsize('datos/test.csv')} bytes")
    else:
        raise FileNotFoundError("test.csv no se encuentra en datos/")
else:
    raise FileNotFoundError("Fallo al descargar el ZIP")

# Cargar el archivo preprocesado
try:
    train_preprocessed = pd.read_csv('datos/train_preprocessed.csv', engine='python', encoding='utf-8')
    print('Columnas en train_preprocessed:', train_preprocessed.columns.tolist())
    print('Tipos de datos:\n', train_preprocessed.dtypes)
except Exception as e:
    print(f"Error al cargar train_preprocessed.csv: {e}")
    raise

# Cargar y preprocesar test.csv
try:
    test = pd.read_csv('datos/test.csv', engine='python', encoding='utf-8')
    print("test.csv cargado correctamente")
    test_ids = test['ID']
except Exception as e:
    print(f"Error al cargar test.csv: {e}")
    raise

# Eliminar columna duplicada si existe
if 'FAMI_TIENEINTERNET.1' in test.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1")
    test = test.drop('FAMI_TIENEINTERNET.1', axis=1)

# Estandarizar valores
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')

# Manejar valores faltantes
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no pagó matrícula')
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].fillna('0')
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].fillna('No sabe')
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].fillna('No sabe')
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].fillna('No')
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].fillna('No')
test['FAMI_TIENELAVADORA'] = test['FAMI_TIENELAVADORA'].fillna('No')
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].fillna('No')
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].fillna('No')
test['ESTU_PRIVADO_LIBERTAD'] = test['ESTU_PRIVADO_LIBERTAD'].fillna('N')

# Definir mapeo para codificación ordinal
valormatricula_map = {
    'no pagó matrícula': 0,
    'menos de 500 mil': 1,
    'entre 500 mil y menos de 1 millón': 2,
    'entre 1 millón y menos de 2.5 millones': 3,
    'entre 2.5 millones y menos de 4 millones': 4,
    'entre 4 millones y menos de 5.5 millones': 5,
    'entre 5.5 millones y menos de 7 millones': 6,
    'más de 7 millones': 7
}
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)

horastrabajo_map = {
    '0': 0,
    'menos de 10 horas': 1,
    'entre 11 y 20 horas': 2,
    'entre 21 y 30 horas': 3,
    'más de 30 horas': 4
}
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)

# Aplicar Label Encoding
label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE']
le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    le.fit(train_preprocessed[col])
    test[col] = test[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<unknown>')
    test[col] = le.transform(test[col])
    le_dict[col] = le

# Aplicar One-Hot Encoding
one_hot_cols = ['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_TIENEINTERNET',
                'FAMI_TIENECOMPUTADOR', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRIVADO_LIBERTAD']
test = pd.get_dummies(test, columns=one_hot_cols, drop_first=True)

# Asegurar que test tenga las mismas columnas que train_preprocessed
missing_cols = set(train_preprocessed.columns) - set(test.columns) - {'ID', 'RENDIMIENTO_GLOBAL'}
for col in missing_cols:
    test[col] = 0
test_preprocessed = test[train_preprocessed.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1).columns]

# Normalizar columnas numéricas
num_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
scaler = MinMaxScaler()
scaler.fit(train_preprocessed[num_cols])
test_preprocessed[num_cols] = scaler.transform(test_preprocessed[num_cols])

# Eliminar columnas duplicadas
test_preprocessed = test_preprocessed.loc[:, ~test_preprocessed.columns.duplicated()]

# Verificar datos
print('Columnas en test_preprocessed:', test_preprocessed.columns.tolist())
print('Tipos de datos en test_preprocessed:\n', test_preprocessed.dtypes)
for col in test_preprocessed.columns:
    if test_preprocessed[col].dtype == 'object':
        print(f"Columna con strings: {col}, valores: {test_preprocessed[col].unique()}")

# Separar características y variable objetivo
X_03 = train_preprocessed.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y_03 = train_preprocessed['RENDIMIENTO_GLOBAL']

# Convertir etiquetas categóricas a numéricas para XGBoost
label_encoder_y = LabelEncoder()
y_03 = label_encoder_y.fit_transform(y_03)

# Verificar datos de entrada
print('Tipos de datos en X_03:\n', X_03.dtypes)
for col in X_03.columns:
    if X_03[col].dtype == 'object':
        print(f"Columna con strings en X_03: {col}, valores: {X_03[col].unique()}")

# Dividir en conjunto de entrenamiento y validación
X_train_03, X_val_03, y_train_03, y_val_03 = train_test_split(X_03, y_03, test_size=0.2, random_state=42, stratify=y_03)

print('Tamaño del conjunto de entrenamiento (03):', X_train_03.shape)
print('Tamaño del conjunto de validación (03):', X_val_03.shape)

# Entrenar modelo XGBoost con parámetros fijos para velocidad
start_time = time.time()
xgb = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_train_03, y_train_03)
print(f"Tiempo de entrenamiento de XGBoost: {(time.time() - start_time) / 60:.2f} minutos")

# Evaluar en conjunto de validación
y_pred_03 = xgb.predict(X_val_03)
print('Precisión en validación (03):', accuracy_score(y_val_03, y_pred_03))
print('Reporte de clasificación (03):\n', classification_report(y_val_03, y_pred_03))

# Generar predicciones para test.csv
test_pred_03 = xgb.predict(test_preprocessed)

# Convertir predicciones numéricas a etiquetas categóricas
test_pred_03 = label_encoder_y.inverse_transform(test_pred_03)

# Crear archivo de sumisión
submission_03 = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': test_pred_03})
submission_03.to_csv('submission_xgboost.csv', index=False)
print('Archivo de sumisión guardado como submission_xgboost.csv')

Por favor, sube el archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente
udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Datos extraídos en datos/
test.csv encontrado, tamaño: 59185250 bytes
Columnas en train_preprocessed: ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_EDUCACIONPADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'FAMI_ESTRATOVIVIENDA_estrato 2', 'FAMI_ESTRATOVIVIENDA_estrato 3', 'FAMI_ESTRATOVIVIENDA_estrato 4', 'FAMI_ESTRATOVIVIENDA_estrato 5', 'FAMI_ESTRATOVIVIENDA_estrato 6', 'FAMI_ESTRATOVIVIENDA_sin estrato', 'FAMI_EDUCACIONMADRE_Educación profesional incompleta', 'FAMI_EDUCACIONMADRE_Ninguno', 'FAMI_EDUCACIONMADRE_No Aplica', 'FAMI_EDUCACIONMADRE_No sabe', 'FAMI_EDUCACIONMADRE_Postgrado', 'FAMI_EDUCACIONMADRE_Primaria completa', 'FAMI_EDUCACIONMADRE_Primaria incompleta', 'FAMI

/tmp/ipython-input-2-2473914720.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preprocessed[num_cols] = scaler.transform(test_preprocessed[num_cols])


Columnas en test_preprocessed: ['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_EDUCACIONPADRE', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'FAMI_ESTRATOVIVIENDA_estrato 2', 'FAMI_ESTRATOVIVIENDA_estrato 3', 'FAMI_ESTRATOVIVIENDA_estrato 4', 'FAMI_ESTRATOVIVIENDA_estrato 5', 'FAMI_ESTRATOVIVIENDA_estrato 6', 'FAMI_ESTRATOVIVIENDA_sin estrato', 'FAMI_EDUCACIONMADRE_Educación profesional incompleta', 'FAMI_EDUCACIONMADRE_Ninguno', 'FAMI_EDUCACIONMADRE_No Aplica', 'FAMI_EDUCACIONMADRE_No sabe', 'FAMI_EDUCACIONMADRE_Postgrado', 'FAMI_EDUCACIONMADRE_Primaria completa', 'FAMI_EDUCACIONMADRE_Primaria incompleta', 'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa', 'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta', 'FAMI_EDUCACIONMADRE_Técnica o tecnológica completa', 'FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta', 'FAMI_TIENEINTERNET_Si', 'FAMI_TIENECOMPUTADOR_Si', 'FAMI_TIENELAVADORA_Si', 'FAMI

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:09:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Tiempo de entrenamiento de XGBoost: 0.73 minutos
Precisión en validación (03): 0.42454151624548736
Reporte de clasificación (03):
               precision    recall  f1-score   support

           0       0.53      0.63      0.57     35124
           1       0.45      0.56      0.50     34597
           2       0.32      0.24      0.28     34324
           3       0.33      0.27      0.29     34455

    accuracy                           0.42    138500
   macro avg       0.41      0.42      0.41    138500
weighted avg       0.41      0.42      0.41    138500

Archivo de sumisión guardado como submission_xgboost.csv
